In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

from multiprocessing import Pool
from functools import partial

from pathlib import Path
from tqdm import tqdm

from scipy.stats import pearsonr
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error

from itertools import product
import warnings

In [3]:
ds = xr.open_dataset('~/scratch/spicy/SnowEx-Data/Banner_2021-03-15.nc')

In [16]:
ds.interp_like(ds.coarsen(x = 100, y = 100, boundary = 'pad').mean())

<xarray.Dataset>
Dimensions:         (band: 3, time: 97, y: 2, x: 2)
Coordinates:
  * band            (band) object 'VV' 'VH' 'inc'
    relative_orbit  (time) float64 71.0 93.0 20.0 71.0 ... 93.0 20.0 71.0 93.0
    absolute_orbit  (time) float64 3.374e+04 3.376e+04 ... 2.617e+04 2.619e+04
    projection      int64 0
  * x               (x) float64 -115.2 -115.1
  * y               (y) float64 44.29 44.22
  * time            (time) datetime64[ns] 2020-08-03T13:49:39 ... 2021-03-27T...
    flight_dir      (time) object 'descending' 'ascending' ... 'ascending'
    platform        (time) object 'S1A' 'S1A' 'S1B' 'S1B' ... 'S1B' 'S1B' 'S1B'
Data variables: (12/17)
    spatial_ref     int64 0
    s1              (time, band, y, x) float64 -9.258 -9.009 nan ... nan 0.4394
    ims             (time, y, x) float64 2.0 2.0 2.0 2.0 2.0 ... 4.0 4.0 4.0 4.0
    fcf             (y, x) float64 0.71 0.865 0.595 0.76
    deltaCR         (time, y, x) float64 nan nan nan nan ... 2.957 nan -3.537
    deltaVV         (time, y, x) float64 nan nan nan nan ... -0.2599 nan 1.446
    ...              ...
    freeze_flag     (time, y, x) float64 1.0 1.0 1.0 1.0 ... 0.25 0.0 1.0 0.0
    wet_snow        (time, y, x) float64 0.0 0.0 0.0 0.0 0.0 ... 0.5 1.0 1.0 1.0
    perma_wet       (time, y, x) float64 0.0 0.0 0.0 0.0 ... 0.0 0.875 0.0
    lidar-sd        (y, x) float64 1.376 1.282 1.004 nan
    lidar-vh        (y, x) float64 9.925 2.921 0.1736 nan
    lidar-dem       (y, x) float64 2.168e+03 2.257e+03 1.79e+03 nan
Attributes: (12/16)
    AREA_OR_POINT:      Point
    TIFFTAG_DATETIME:   2023:08:31 00:47:59
    TIFFTAG_SOFTWARE:   Created with GAMMA Software www.gamma-rs.ch data2geot...
    scale_factor:       1.0
    add_offset:         0.0
    resolution:         90
    ...                 ...
    job_name:           spicy_USIDBS_2021-03-29
    bounds:             [-115.28143275   44.19225956 -115.08677234   44.34416...
    site:               Banner
    site_abbrev:        USIDBS
    lidar-flight-time:  2021-03-15
    processing-date:    2023-08-30 23:16:43.458130

In [17]:
import rioxarray as rxa
from rasterio.enums import Resampling

In [21]:
ds['snow_depth'].rio.write_crs('EPSG:4326').transpose('time', 'y', 'x').rio.reproject_match(ds['snow_depth'].rio.write_crs('EPSG:4326').transpose('time', 'y', 'x').coarsen(x = 10, y = 10, boundary = 'pad').mean(), resampling = Resampling.average)

<xarray.DataArray 'snow_depth' (time: 97, y: 16, x: 20)>
array([[[0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ]],

       [[0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
...
        [       nan,        nan,        nan, ...,        nan,
         0.88003279,        nan],
        [1.90050825,        nan,        nan, ...,        nan,
         1.14610094,        nan],
        [2.33289112, 1.61140277,        nan, ..., 1.51860554,
         1.00307589,        nan]],

       [[1.98700994,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        ...,
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan, 3.60346376]]])
Coordinates:
  * x               (x) float64 -115.3 -115.3 -115.3 ... -115.1 -115.1 -115.1
  * y               (y) float64 44.34 44.33 44.32 44.31 ... 44.21 44.2 44.19
  * time            (time) datetime64[ns] 2020-08-03T13:49:39 ... 2021-03-27T...
    absolute_orbit  (time) float64 3.374e+04 3.376e+04 ... 2.617e+04 2.619e+04
    platform        (time) object 'S1A' 'S1A' 'S1B' 'S1B' ... 'S1B' 'S1B' 'S1B'
    flight_dir      (time) object 'descending' 'ascending' ... 'ascending'
    relative_orbit  (time) float64 71.0 93.0 20.0 71.0 ... 93.0 20.0 71.0 93.0
    projection      int64 0
    spatial_ref     int64 0

In [19]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

from multiprocessing import Pool
from functools import partial

from pathlib import Path
from tqdm import tqdm

from scipy.stats import pearsonr
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error

from itertools import product
import warnings

# from tqdm.contrib.itertools import product

def get_stats(a, b):
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", message="An input array is constant")
        r, p = pearsonr(a, b)
    error = mae(a, b)
    rmse = mean_squared_error(a, b, squared=False)
    bias = np.mean(a - b)
    return r, error, rmse, bias


def make_stat_da(tree_idx_low, tree_idx_high, loc_fp):
    
    print(f'Starting {loc_fp.stem}...')

    stems= [f.stem for f in list(loc_fp.glob('*_*_*.npy'))]

    A = np.unique([float(s.split('_')[0]) for s in stems])
    B = np.unique([float(s.split('_')[1]) for s in stems])
    C = np.unique([float(s.split('_')[2]) for s in stems])

    res = np.zeros((1, len(A), len(B), len(C)))

    da = xr.DataArray(res, coords = [[loc_fp.stem], A, B, C], dims = ['location', 'A', 'B','C'], name = 'pearsonr')
    res_ds = xr.merge([da, da.copy().rename('mae'), da.copy().rename('rmse'), da.copy().rename('bias')])

    lidar_orig = np.load(loc_fp.joinpath('lidar.npy'))
    elev = np.load(loc_fp.joinpath('elev.npy'))
    trees = np.load(loc_fp.joinpath('trees.npy'))

    idx = (trees <= tree_idx_high) & (elev > 0) & (trees >= tree_idx_low)
    print(idx)
    print(A)

    if 'Cottonwood' in loc_fp.stem:
        print(f"Size of datarray: {res_ds['pearsonr'].data.shape}")
        print(f'Used fraction: {np.sum(idx)/elev.size}')

    for a, b, c in product(A, B, C):
        sds_orig = np.load(loc_fp.joinpath(f'{a}_{b}_{c}.npy'))
        combo = np.vstack([lidar_orig, sds_orig])
        combo = combo.T[idx].T

        sds, lidar = combo
        r, mean_error, rmse, bias = get_stats(lidar, sds)
        res_ds['pearsonr'].loc[dict(location = loc_fp.stem, A = a, B = b, C = c)] = r
        res_ds['mae'].loc[dict(location = loc_fp.stem, A = a, B = b, C = c)] = mean_error
        res_ds['rmse'].loc[dict(location = loc_fp.stem, A = a, B = b, C = c)] = rmse
        res_ds['bias'].loc[dict(location = loc_fp.stem, A = a, B = b, C = c)] = bias
    
    for dv in res_ds.data_vars:
        res_ds[dv] = res_ds[dv].astype(float)
    res_ds.to_netcdf(loc_fp.parent.joinpath('stats_ncs', loc_fp.stem + f'{tree_idx_low}_{tree_idx_high}_stats.nc'))

    print(f'Finishing {loc_fp.stem}!')


param_fp = Path('/bsuhome/zacharykeskinen/scratch/spicy/param_npys')

param_fp.joinpath('stats_ncs').mkdir(exist_ok = True)


out_fp = Path('/bsuhome/zacharykeskinen/scratch/spicy/param_stats.nc')

if out_fp.exists():
    print('Already exists...')

# pool = Pool()
    
# pool.map(partial(make_stat_da, 0, 1), param_fp.glob('*_*-*-*'))
# das = [xr.open_dataset(fp) for fp in param_fp.joinpath('stats_ncs').glob('*0_1_stats.nc')]
# ds = xr.merge(das)
# ds.to_netcdf(param_fp.joinpath('param_stats_all.nc'))

# pool = Pool()
# pool.map(partial(make_stat_da, 0, 0.25), param_fp.glob('*_*-*-*'))
# das = [xr.open_dataset(fp) for fp in param_fp.joinpath('stats_ncs').glob('*0_0.25_stats.nc')]
# ds = xr.merge(das)
# ds.to_netcdf(param_fp.joinpath('param_stats_low_fcf.nc'))

# pool = Pool()
# pool.map(partial(make_stat_da, 0.75, 1), param_fp.glob('*_*-*-*'))
# das = [xr.open_dataset(fp) for fp in param_fp.joinpath('stats_ncs').glob('*0_0.25_stats.nc')]
# ds = xr.merge(das)
# ds.to_netcdf(param_fp.joinpath('param_stats_high_fcf.nc'))

# # locs = list(param_fp.glob('*'))
# # locs = [l.stem for l in locs]

In [20]:
make_stat_da(0, 1, next(param_fp.glob('*_*-*-*')))

Starting Mores_2021-03-15...
[ True  True  True ...  True  True  True]
[1.  1.1 1.2 1.3 1.4 1.5 1.6 1.7 1.8 1.9 2.  2.1 2.2 2.3 2.4 2.5 2.6 2.7
 2.8 2.9 3. ]


KeyboardInterrupt: 

In [11]:
param_fp

PosixPath('/bsuhome/zacharykeskinen/scratch/spicy/param_npys')